In [16]:
import pandas as pd
import os
import math as math
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pickle

In [17]:
df = pd.read_csv('dataset/use/content_df.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
df.dtypes

Title             object
description       object
authors           object
image             object
previewLink       object
publisher         object
publishedDate     object
infoLink          object
categories        object
ratingsCount     float64
dtype: object

In [ ]:
features = df[['Title', 'description', 'authors', 'publishedDate', 'categories', 'publisher']]
features

,Title,description,authors,publishedDate,categories,publisher
0,The Rabbi's Cat,Gaining the ability to speak after swallowing ...,['Joann Sfar'],2005,['Comics & Graphic Novels'],Pantheon
1,Thomas Paine : Collected Writings : Common Sen...,Thomas Paine was the impassioned democratic vo...,['Thomas Paine'],1995-03-01,['History'],Library of America
2,Lenin's Tomb: The Last Days of the Soviet Empire,Winner of the Pulitzer Prize One of the Best B...,['David Remnick'],2014-04-02,['History'],Vintage
3,The Triumph of the Moon: A History of Modern P...,'a brilliant history' The Sunday Times 'makes ...,['Ronald Hutton'],2019-10-10,"['Body, Mind & Spirit']",Oxford University Press
4,Lifetimes: The Beautiful Way to Explain Death ...,"When the death of a relative, a friend, or a p...",['Bryan Mellonie'],2009-09-16,['Family & Relationships'],Bantam
...,...,...,...,...,...,...
995,Time Enough for Love: The Lives of Lazarus Lon...,Follows Lazarus Long through a vast and magnif...,['Robert A. Heinlein'],2021-10-05,['Fiction'],Penguin
996,Marriage Fitness: 4 Steps to Building & Mainta...,Revolutionary step by step system marriage suc...,['Mort Fertel'],2004,['Religion'],Marriage Max Incorporated
997,Push: A Novel,A Good Morning America Book Club Pick | A New ...,['Ashley Audrain'],2021-01-05,['Fiction'],Penguin
998,The World According to Humphrey,"Humphrey, pet hamster at Longfellow School, le...",['Betty G. Birney'],2005,['Juvenile Fiction'],Penguin


In [ ]:
features['publishedYear'] = features['publishedDate'].str[:4]
features.drop(columns=['publishedDate'])

C:\Users\Bagas\AppData\Local\Temp\ipykernel_36840\1397074523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['publishedYear'] = features['publishedDate'].str[:4]


,Title,description,authors,categories,publisher,publishedYear
0,The Rabbi's Cat,Gaining the ability to speak after swallowing ...,['Joann Sfar'],['Comics & Graphic Novels'],Pantheon,2005
1,Thomas Paine : Collected Writings : Common Sen...,Thomas Paine was the impassioned democratic vo...,['Thomas Paine'],['History'],Library of America,1995
2,Lenin's Tomb: The Last Days of the Soviet Empire,Winner of the Pulitzer Prize One of the Best B...,['David Remnick'],['History'],Vintage,2014
3,The Triumph of the Moon: A History of Modern P...,'a brilliant history' The Sunday Times 'makes ...,['Ronald Hutton'],"['Body, Mind & Spirit']",Oxford University Press,2019
4,Lifetimes: The Beautiful Way to Explain Death ...,"When the death of a relative, a friend, or a p...",['Bryan Mellonie'],['Family & Relationships'],Bantam,2009
...,...,...,...,...,...,...
995,Time Enough for Love: The Lives of Lazarus Lon...,Follows Lazarus Long through a vast and magnif...,['Robert A. Heinlein'],['Fiction'],Penguin,2021
996,Marriage Fitness: 4 Steps to Building & Mainta...,Revolutionary step by step system marriage suc...,['Mort Fertel'],['Religion'],Marriage Max Incorporated,2004
997,Push: A Novel,A Good Morning America Book Club Pick | A New ...,['Ashley Audrain'],['Fiction'],Penguin,2021
998,The World According to Humphrey,"Humphrey, pet hamster at Longfellow School, le...",['Betty G. Birney'],['Juvenile Fiction'],Penguin,2005


In [ ]:
features['content'] = features['description'] + ' ' + features['authors'] + ' ' + features['categories'] + 'publisher'

C:\Users\Bagas\AppData\Local\Temp\ipykernel_36840\1305833510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['content'] = features['description'] + ' ' + features['authors'] + ' ' + features['categories'] + 'publisher'


In [ ]:
features

,Title,description,authors,publishedDate,categories,publisher,publishedYear,content
0,The Rabbi's Cat,Gaining the ability to speak after swallowing ...,['Joann Sfar'],2005,['Comics & Graphic Novels'],Pantheon,2005,Gaining the ability to speak after swallowing ...
1,Thomas Paine : Collected Writings : Common Sen...,Thomas Paine was the impassioned democratic vo...,['Thomas Paine'],1995-03-01,['History'],Library of America,1995,Thomas Paine was the impassioned democratic vo...
2,Lenin's Tomb: The Last Days of the Soviet Empire,Winner of the Pulitzer Prize One of the Best B...,['David Remnick'],2014-04-02,['History'],Vintage,2014,Winner of the Pulitzer Prize One of the Best B...
3,The Triumph of the Moon: A History of Modern P...,'a brilliant history' The Sunday Times 'makes ...,['Ronald Hutton'],2019-10-10,"['Body, Mind & Spirit']",Oxford University Press,2019,'a brilliant history' The Sunday Times 'makes ...
4,Lifetimes: The Beautiful Way to Explain Death ...,"When the death of a relative, a friend, or a p...",['Bryan Mellonie'],2009-09-16,['Family & Relationships'],Bantam,2009,"When the death of a relative, a friend, or a p..."
...,...,...,...,...,...,...,...,...
995,Time Enough for Love: The Lives of Lazarus Lon...,Follows Lazarus Long through a vast and magnif...,['Robert A. Heinlein'],2021-10-05,['Fiction'],Penguin,2021,Follows Lazarus Long through a vast and magnif...
996,Marriage Fitness: 4 Steps to Building & Mainta...,Revolutionary step by step system marriage suc...,['Mort Fertel'],2004,['Religion'],Marriage Max Incorporated,2004,Revolutionary step by step system marriage suc...
997,Push: A Novel,A Good Morning America Book Club Pick | A New ...,['Ashley Audrain'],2021-01-05,['Fiction'],Penguin,2021,A Good Morning America Book Club Pick | A New ...
998,The World According to Humphrey,"Humphrey, pet hamster at Longfellow School, le...",['Betty G. Birney'],2005,['Juvenile Fiction'],Penguin,2005,"Humphrey, pet hamster at Longfellow School, le..."


In [ ]:
stopword = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower() 
    text = re.sub(r'\d+', '', text) 
    text = re.sub(r'\b\w{1,2}\b', '', text)  
    text = re.sub(r'[^\w\s]', '', text) 
    text = re.sub(r'\s+', ' ', text).strip() 
    text = ' '.join(word for word in text.split() if word not in stopword)
    return text

features['clean_content'] = features['content'].apply(preprocess_text)
features.set_index(features['Title'], inplace=True)
features

C:\Users\Bagas\AppData\Local\Temp\ipykernel_36840\1719039498.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['clean_content'] = features['content'].apply(preprocess_text)


,Title,description,authors,publishedDate,categories,publisher,publishedYear,content,clean_content
Title,,,,,,,,,
The Rabbi's Cat,The Rabbi's Cat,Gaining the ability to speak after swallowing ...,['Joann Sfar'],2005,['Comics & Graphic Novels'],Pantheon,2005,Gaining the ability to speak after swallowing ...,gaining ability speak swallowing parakeet rabb...
"Thomas Paine : Collected Writings : Common Sense / The Crisis / Rights of Man / The Age of Reason / Pamphlets, Articles, and Letters (Library of America)",Thomas Paine : Collected Writings : Common Sen...,Thomas Paine was the impassioned democratic vo...,['Thomas Paine'],1995-03-01,['History'],Library of America,1995,Thomas Paine was the impassioned democratic vo...,thomas paine impassioned democratic voice age ...
Lenin's Tomb: The Last Days of the Soviet Empire,Lenin's Tomb: The Last Days of the Soviet Empire,Winner of the Pulitzer Prize One of the Best B...,['David Remnick'],2014-04-02,['History'],Vintage,2014,Winner of the Pulitzer Prize One of the Best B...,winner pulitzer prize one best books year new ...
The Triumph of the Moon: A History of Modern Pagan Witchcraft,The Triumph of the Moon: A History of Modern P...,'a brilliant history' The Sunday Times 'makes ...,['Ronald Hutton'],2019-10-10,"['Body, Mind & Spirit']",Oxford University Press,2019,'a brilliant history' The Sunday Times 'makes ...,brilliant history sunday times makes riveting ...
Lifetimes: The Beautiful Way to Explain Death to Children,Lifetimes: The Beautiful Way to Explain Death ...,"When the death of a relative, a friend, or a p...",['Bryan Mellonie'],2009-09-16,['Family & Relationships'],Bantam,2009,"When the death of a relative, a friend, or a p...",death relative friend pet happens happen help ...
...,...,...,...,...,...,...,...,...,...
Time Enough for Love: The Lives of Lazarus Long (Part 1 of 2 parts)(Library Edition),Time Enough for Love: The Lives of Lazarus Lon...,Follows Lazarus Long through a vast and magnif...,['Robert A. Heinlein'],2021-10-05,['Fiction'],Penguin,2021,Follows Lazarus Long through a vast and magnif...,follows lazarus long vast magnificent timescap...
Marriage Fitness: 4 Steps to Building & Maintaining Phenomenal Love,Marriage Fitness: 4 Steps to Building & Mainta...,Revolutionary step by step system marriage suc...,['Mort Fertel'],2004,['Religion'],Marriage Max Incorporated,2004,Revolutionary step by step system marriage suc...,revolutionary step step system marriage succes...
Push: A Novel,Push: A Novel,A Good Morning America Book Club Pick | A New ...,['Ashley Audrain'],2021-01-05,['Fiction'],Penguin,2021,A Good Morning America Book Club Pick | A New ...,good morning america book club pick new york t...


In [ ]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), stop_words='english')
tfidf = tf.fit_transform(features['clean_content'])
cos_sim = cosine_similarity(tfidf, tfidf)
cos_sim

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 1.00145667e-02, ...,
        7.94348452e-03, 0.00000000e+00, 2.78490616e-03],
       [0.00000000e+00, 1.00145667e-02, 1.00000000e+00, ...,
        6.97372655e-03, 0.00000000e+00, 2.14026870e-03],
       ...,
       [0.00000000e+00, 7.94348452e-03, 6.97372655e-03, ...,
        1.00000000e+00, 6.31633966e-04, 2.90896351e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        6.31633966e-04, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.78490616e-03, 2.14026870e-03, ...,
        2.90896351e-03, 0.00000000e+00, 1.00000000e+00]])

In [ ]:
def recomendation(name,cos_sim=cos_sim, top_n=10) :
#     if name not in features.index :
#         return f"Books '{name}' not found in database."
    idx = features.index.get_loc(name)
    score = pd.Series(cos_sim[idx]).sort_values(ascending=False)
    top_score = list(score.iloc[1:top_n+1].index)
    
    recommended_books = features.iloc[top_score]['Title'].unique()
    
    return pd.DataFrame(recommended_books, columns=['Recommended Books'])

In [ ]:
recomendation(name="Great Expectations")

,Recommended Books
0,The Mystery of Edwin Drood
1,Martin Chuzzlewit (Penguin Classics)
2,"Moby-Dick: or, The Whale (Penguin Classics Del..."
3,Pajama Time!
4,O Pioneers! (Penguin Classics)
5,The Charles F. Stanley Life Principles Bible (...
6,Reading Like a Writer: A Guide for People Who ...
7,George Orwell: Animal Farm-Nineteen Eighty-Four
8,A Lady of His Own (Bastion Club)
9,Manson in His Own Words: The Shocking Confessi...


In [ ]:
if not os.path.exists('models') :
    os.makedirs('models', exist_ok=True)

file_path = os.path.join('models', 'content.pkl')
with open(file_path, 'wb') as file :
    pickle.dump({'cos_sim' : cos_sim, 'tfidf' : tf}, file)
    
print(f'Model saved succesfully in : {file_path}')

Model saved succesfully in : models\content.pkl
